In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../../..")

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
from librep.datasets.har.loaders import (
    MotionSense_BalancedView20HZ
)

2023-01-04 11:34:39.836476: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-04 11:34:39.983625: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-04 11:34:39.983648: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-04 11:34:40.746712: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [5]:
from librep.transforms.topo_ae import (
    TopologicalDimensionalityReduction,
    CustomTopoDimRedTransform
)
from librep.transforms import UMAP
from sklearn.manifold import TSNE
from experiments.Experiment_utils import *
from librep.datasets.multimodal import TransformMultiModalDataset
from librep.transforms.fft import FFT
from tqdm.notebook import tqdm

In [6]:
# MotionSense Loader
loader = MotionSense_BalancedView20HZ(
    root_dir="../../../data/views/MotionSense/balanced_view_20Hz_with_gravity_9.81_acc_standard", 
    download=False
)

# Print the readme (optional)
# loader.print_readme()

In [7]:
# Load the dataset
# If concat_train_validation is true, return a tuple (train+validation, test)
train_val, test = loader.load(concat_train_validation=True, label=loader.standard_label)
train_val, test

(PandasMultiModalDataset: samples=4020, features=360, no. window=6, label_columns='standard activity code',
 PandasMultiModalDataset: samples=1020, features=360, no. window=6, label_columns='standard activity code')

In [8]:
train_HD = np.array(train_val[:][0])
train_Y = np.array(train_val[:][1])
test_HD = np.array(test[:][0])
test_Y = np.array(test[:][1])

In [9]:
print(train_HD.shape, train_Y.shape, test_HD.shape, test_Y.shape)

(4020, 360) (4020,) (1020, 360) (1020,)


# Aplicar FFT

In [10]:
fft_transform = FFT(centered = True)
transformer = TransformMultiModalDataset(
    transforms=[fft_transform],
    new_window_name_prefix="fft."
)

In [11]:
train_dataset_fft = transformer(train_val)
test_dataset_fft = transformer(test)

In [12]:
print(train_dataset_fft.X.shape)
print(test_dataset_fft.X.shape)

(4020, 180)
(1020, 180)


In [13]:
print(np.max(train_dataset_fft.X))

797.3368276806895


In [14]:
train_HD = train_dataset_fft.X
train_LD = None
train_Y = train_dataset_fft.y
test_HD = test_dataset_fft.X
test_LD = None
test_Y = test_dataset_fft.y

In [15]:
print(train_HD.shape, train_Y.shape, test_HD.shape, test_Y.shape)

(4020, 180) (4020,) (1020, 180) (1020,)


## Preparing for evaluation

### Reporter

In [16]:
reporter = ClassificationReport(
    use_accuracy=True,
    use_f1_score=True,
    use_classification_report=False,
    use_confusion_matrix=False,
    plot_confusion_matrix=False
)

### Defining experiments

In [17]:
experiments = [
    {
        'estimator_tag': 'RF',
        'estimator': RandomForestClassifier,
        'estimator_creation_kwags': {'n_estimators':100},
        'num_runs': 10
    },
    {
        'estimator_tag': 'SVC', 
        'estimator': SVC,
        'estimator_creation_kwags': {'C': 3.0, 'kernel': "rbf"},
        'num_runs': 1
    },
    {
        'estimator_tag': 'KNN',
        'estimator': KNeighborsClassifier,
        'estimator_creation_kwags': {'n_neighbors' :1},
        'num_runs': 1
    }
]

## Reading model

In [18]:
import pickle
model_name = 'ConvolutionalAutoencoder_custom_dim3'
model_lambdas = [0.01, 0.1, 0, 1, 10, 100]
model_startdim = 180
model_latentdim = 10
model_epochs = list(range(250, 5001, 250))
model_ids = list(range(15, 35))
model_ids = [14]
for id_used in tqdm(model_ids):
    for lambda_used in tqdm(model_lambdas):
        experiments_result = {
            'model_name':model_name,
            'model_lambda': lambda_used,
            'results': []
        }
        for epoch_used in tqdm(model_epochs):
            filename = 'data/{}_{}_180-10_5000_{}.pkl'.format(model_name, lambda_used, id_used)
            filehandler = open(filename, 'rb')
            model_reducer = pickle.load(filehandler)
            filehandler.close()
            model_reducer.partial_load(epoch_used)
            train_LD = model_reducer.transform(train_HD)
            test_LD = model_reducer.transform(test_HD)
            train_LD = np.reshape(train_LD, (-1, model_latentdim))
            test_LD = np.reshape(test_LD, (-1, model_latentdim))
            eval_obj = {}
            for experiment in experiments:
                simple_workflow = SimpleTrainEvalWorkflow(
                    estimator=experiment['estimator'],
                    estimator_creation_kwags=experiment['estimator_creation_kwags'],
                    evaluator=reporter
                )
                multi_run_experiment = MultiRunWorkflow(
                    workflow=simple_workflow,
                    num_runs=experiment['num_runs']
                )
                result = multi_run_experiment(
                    ArrayMultiModalDataset(train_LD, train_Y, window_slices=[]),
                    ArrayMultiModalDataset(test_LD, test_Y, window_slices=[])
                )
                eval_obj[experiment['estimator_tag']+'-ACC'] = [run['result'][0]['accuracy'] for run in result['runs']]
                eval_obj[experiment['estimator_tag']+'-F1'] = [run['result'][0]['f1 score (weighted)'] for run in result['runs']]
            experiments_result['results'].append((epoch_used, eval_obj))
        filename_to_save = 'data_eval/{}_{}_180-10_5000_{}.acc_f1.pkl'.format(model_name, lambda_used, id_used)
        filehandler = open(filename_to_save, 'wb')
        pickle.dump(experiments_result, filehandler)
        filehandler.close()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [19]:
filename = 'data_eval/ConvolutionalAutoencoder_custom_dim3_0.01_180-10_5000_0.acc_f1.pkl'
filehandler = open(filename, 'rb')
data = pickle.load(filehandler)
print(data['results'][1])
filehandler.close()

(500, {'RF-ACC': [0.9009803921568628, 0.9009803921568628, 0.9019607843137255, 0.9019607843137255, 0.907843137254902, 0.9049019607843137, 0.907843137254902, 0.9009803921568628, 0.903921568627451, 0.8980392156862745], 'RF-F1': [0.901185828761331, 0.901270630142591, 0.9022598076500604, 0.9023214169579444, 0.9080119615745122, 0.9051018758390784, 0.9081805599506075, 0.9011492171870802, 0.9040893654799241, 0.8985282726434471], 'SVC-ACC': [0.9019607843137255], 'SVC-F1': [0.9026034835419132], 'KNN-ACC': [0.8607843137254902], 'KNN-F1': [0.8604010709913317]})


In [20]:
assert 1==0

AssertionError: 

In [ ]:
import pickle

# Variables to reuse
save_ids = range(5)
model_name = 'ConvolutionalAutoencoder_custom_dim3'
model_dim = 10
model_epc = 500
model_lams = [100]

for model_lam in model_lams:
    for save_id in save_ids:
        filename = 'data/{}_{}_{}_{}_{}.pkl'.format(model_name, model_lam, model_dim, model_epc, save_id)
        filehandler = open(filename, 'rb')
        model_reducer = pickle.load(filehandler)
        filehandler.close()
        train_LD = model_reducer.transform(train_HD)
        test_LD = model_reducer.transform(test_HD)
        train_LD = np.reshape(train_LD, (-1, model_dim))
        print('TRAIN LD RESHAPED', train_LD.shape)
        test_LD = np.reshape(test_LD, (-1, model_dim))
        print('TEST LD RESHAPED', test_LD.shape)
        experiments_result = run_experiments(train_HD, train_LD, train_Y, test_HD, test_LD, test_Y)
        filename_to_save = 'data/{}_{}_{}_{}_{}.acc_f1.pkl'.format(model_name, model_lam, model_dim, model_epc, save_id)
        filehandler = open(filename_to_save, 'wb')
        pickle.dump(experiments_result, filehandler)
        filehandler.close()